In [1]:
using Random
using Distributions
using LinearAlgebra
using DataFrames
using StatsBase
using PDMats
using CSV
using Dates
using JuMP
using Ipopt
import SpecialFunctions: loggamma
import MathOptInterface as MOI

In [2]:
const EPS = 1e-8
safe_beta(α, β) = Beta(max(α, EPS), max(β, EPS))

const LOG_EPS = 1e-200
safe_log(x) = log(max(x, LOG_EPS))

const GAMMA_EPS = 1e-8
function safe_gamma(shape, rate)
    α = max(shape, GAMMA_EPS)
    r = max(rate,  GAMMA_EPS)
    θ = 1/r
    θ = max(θ, GAMMA_EPS)
    return Gamma(α, θ)
end

safe_bernoulli(p::Real) = Bernoulli(clamp(p, 0.0, 1.0))

function safe_pdmat(A::AbstractMatrix{<:Real}; eps0=1e-8, maxiter=5)
    S = Symmetric((A + A')/2)
    for i in 0:maxiter
        try
            return PDMat(Float64.(S + (eps0*i)*I))
        catch e
            if !(e isa PosDefException) || i==maxiter
                rethrow(e)
            end
        end
    end
end

const DIR_EPS = 1e-8
function safe_dirichlet(α::AbstractVector{<:Real})
    a = max.(Float64.(α), DIR_EPS)
    s = sum(a)
    x = rand(Dirichlet(a))
    if any(!isfinite, x) || any(<(0.0), x)
        return a ./ s
    end
    sx = sum(x)
    return (isfinite(sx) && abs(sx - 1.0) ≤ 1e-10) ? x : (a ./ s)
end

safe_dirichlet (generic function with 1 method)

In [3]:
# Data-generating parameters
pi_true = [
  vec([0.5731372264896774 0.3584768992248216 0.01085525821402486 0.05753061607147426]),
  vec([0.0370388230815912 0.889042514556095 0.002007726412896501 0.07191093594941617]),
  vec([0.006318513965246314 0.0071812424323565 0.7291813565982903 0.257318887004104]),
  vec([0.01157929936688799 0.08442420898922519 0.1134931812666756 0.7905033103772089])
]

mu_true = [
  vec([0.006734506602107449 0.003381498351761594 0.01251958486816306 0.007574169914426993]),
  vec([0.0009441833395542966 0.0003530895525309008 -0.001122619477144013 3.041564764696133e-06]),
  vec([-0.0001357129572322458 0.0001789316615930574 -0.0002532572436555386 -0.0005170677905046179]),
  vec([0.001248987177806565 0.001216024923786339 0.0009936349596729377 -0.0002090026527542731])
]

Sigma_true = [
  [
      0.002054506019709262 0.0005749811760326436 0.0005816549429042377 0.0006146245962203936;
      0.0005749811760326436 0.002379851613164497 0.001639530029524073 0.003591993629918802;
      0.0005816549429042377 0.001639530029524073 0.003565602256146811 0.002759543733341633;
      0.0006146245962203936 0.003591993629918802 0.002759543733341633 0.01157320665447896
],
  [
      0.0002717377294919257 5.902118513846123e-05 6.848107986651424e-05 2.509832559253089e-05;
      5.902118513846123e-05 0.0002444336297386139 0.0001685020755143676 0.0001883782584247908;
      6.848107986651424e-05 0.0001685020755143676 0.0005004297863327951 0.0002691805211144379;
      2.509832559253089e-05 0.0001883782584247908 0.0002691805211144379 0.0008924838375103492
],
  [
      0.000402320019330142 0.0001061006439395391 6.300527335193929e-05 6.423870914075368e-05;
      0.0001061006439395391 0.0003125691916858654 0.000169295996800317 0.0001073321464084567;
      6.300527335193929e-05 0.000169295996800317 0.0002752254194025297 8.940431341503119e-05;
      6.423870914075368e-05 0.0001073321464084567 8.940431341503119e-05 0.0001430435998786714
],
  [
      6.386967467667764e-05 1.065760662021473e-05 1.730953121602137e-05 -2.450550198361974e-06;
      1.065760662021473e-05 6.364941106262232e-05 4.037048036193801e-05 1.303416250098312e-05;
      1.730953121602137e-05 4.037048036193801e-05 0.0001441498727498638 4.898473242735126e-05;
      -2.450550198361974e-06 1.303416250098312e-05 4.898473242735126e-05 0.0002539259627920557
]
]

4-element Vector{Matrix{Float64}}:
 [0.002054506019709262 0.0005749811760326436 0.0005816549429042377 0.0006146245962203936; 0.0005749811760326436 0.002379851613164497 0.001639530029524073 0.003591993629918802; 0.0005816549429042377 0.001639530029524073 0.003565602256146811 0.002759543733341633; 0.0006146245962203936 0.003591993629918802 0.002759543733341633 0.01157320665447896]
 [0.0002717377294919257 5.902118513846123e-5 6.848107986651424e-5 2.509832559253089e-5; 5.902118513846123e-5 0.0002444336297386139 0.0001685020755143676 0.0001883782584247908; 6.848107986651424e-5 0.0001685020755143676 0.0005004297863327951 0.0002691805211144379; 2.509832559253089e-5 0.0001883782584247908 0.0002691805211144379 0.0008924838375103492]
 [0.000402320019330142 0.0001061006439395391 6.30052733519393e-5 6.423870914075368e-5; 0.0001061006439395391 0.0003125691916858654 0.000169295996800317 0.0001073321464084567; 6.30052733519393e-5 0.000169295996800317 0.0002752254194025297 8.940431341503119e-5; 6.4238

In [4]:
# Warm-start container for run_pp
Base.@kwdef mutable struct PPState
    S::Vector{Int}
    β::Vector{Float64}
    ξ::Vector{Float64}
    χ::Vector{Float64}
    α::Vector{Float64}
    λ::Vector{Float64}
    π_vec::Vector{Vector{Float64}}
    μ::Vector{Vector{Float64}}
    Σ::Vector{PDMat{Float64}}
end


PPState

In [5]:
function run_pp(Y::Matrix{Float64}, T::Int, H::Int, D::Int; state::Union{Nothing,PPState}=nothing)  
    # Concentration priors
    C1, C2 = 3, 1
    X1, X2 = 3, 2

    # Number of clusters
    K_init   = 2

    # NIW prior hyperparameters
    mu0     = zeros(D)
    kappa0  = 5
    rho0    = D + 2
    Psi0 = safe_pdmat(0.0005 * Matrix{Float64}(I, D, D))

    if state === nothing
        N_warmup  = 1000
        N_sample  = 500

        K = K_init
        S = rand(1:K, T)
    
        # Draw β
        β = safe_dirichlet(ones(K));   β .= β ./ sum(β)
    
        # Preallocate containers
        ξ = Vector{Float64}(undef, K)
        χ = Vector{Float64}(undef, K)
        α = Vector{Float64}(undef, K)
        λ = Vector{Float64}(undef, K)
        π_vec = Vector{Vector{Float64}}(undef, K)
        μ = Vector{Vector{Float64}}(undef, K)
        Σ = Vector{PDMat{Float64}}(undef, K)
    
        δ = PDMat(Matrix{Float64}(I, K, K))

        # Loop over components j = 1:K
        for j in 1:K
            # Draw stick‐breaking parameters
            ξ[j] = rand(safe_beta(X1, X2))
            χ[j] = rand(safe_gamma(C1, C2))

            # Compute concentration parameters
            α[j] = (1 - ξ[j]) * χ[j]
            λ[j] =        ξ[j]  * χ[j]

            # Build the Dirichlet parameter vector for π_j
            δ_j = δ[:, j]
            dir_pars = α[j] .* β .+ λ[j] .* δ_j
            π_vec[j] = safe_dirichlet(dir_pars)
            π_vec[j] .= π_vec[j] ./ sum(π_vec[j])

            # Draw from NIW
            Σ_mat = rand(InverseWishart(rho0, Psi0))
            Σ[j]  = safe_pdmat(Σ_mat)
            μ[j]  = rand(MvNormal(mu0, Σ[j] / kappa0))
        end

        # Count assignments
        m     = zeros(Int, K)
        n_mat = zeros(Int, K, K)
        q_mat = zeros(Int, K, K)
        Q     = zeros(Int, K)

        m[S[1]] += 1
        for t in 2:T
            m[S[t]]             += 1
            n_mat[S[t-1], S[t]] += 1
        end
    else
        N_warmup  = 100
        N_sample  = 500

        K = K_init

        # Re-use the previous iteration’s variables
        S     = copy(state.S)
        β     = copy(state.β)
        ξ     = copy(state.ξ)
        χ     = copy(state.χ)
        α     = copy(state.α)
        λ     = copy(state.λ)
        π_vec = deepcopy(state.π_vec)
        μ     = deepcopy(state.μ)
        Σ     = deepcopy(state.Σ)
    
        δ = PDMat(Matrix{Float64}(I, K, K))

        # Shift left by 1
        S[1:end-1] = S[2:end]

        # Sample the new S[T] from π[S[T-1]]
        s_prev = S[end-1]
        w = copy(π_vec[s_prev][1:K])
        sw = sum(w)
        if sw <= 0 || !isfinite(sw)
            fill!(w, 1.0 / K)
        else
            @. w = w / sw
        end

        # Draw S[T] ~ Categorical(w)
        S[end] = rand(Categorical(w))

        # Count assignments
        m     = zeros(Int, K)
        n_mat = zeros(Int, K, K)
        q_mat = zeros(Int, K, K)
        Q     = zeros(Int, K)

        m[S[1]] += 1
        for t in 2:T
            m[S[t]]             += 1
            n_mat[S[t-1], S[t]] += 1
        end
    end

    # Total number of iterations
    total_iters = N_warmup + N_sample

    # Preallocate prediction containers
    mu_bar      = zeros(Float64, H, D)
    mumuT_bar   = zeros(Float64, H, D, D)
    Sigma_bar   = zeros(Float64, H, D, D)

    for n_iter in 1:total_iters
        # Update latent states & counts
        for t in 1:T
            # Decrement counts for the current state
            m[S[t]] -= 1
            if t > 1
                n_mat[S[t-1], S[t]] -= 1
            end
            if t < T
                n_mat[S[t], S[t+1]] -= 1
            end
            logW = fill(-Inf, length(β))
            for k in 1:length(β)
                if t == 1
                    log_w_prev = safe_log(β[k]) + safe_log(π_vec[k][S[t+1]])
                elseif t == T
                    log_w_prev = safe_log(π_vec[S[t-1]][k])
                else
                    log_w_prev = safe_log(π_vec[S[t-1]][k]) + safe_log(π_vec[k][S[t+1]])
                end
                if isfinite(log_w_prev)
                    resid = Y[t, :] .- μ[k]
                    quad = dot(resid, (Σ[k]) \ resid)
                    logW[k] = log_w_prev - 0.5 * (quad + logdet(Σ[k]))
                end
            end
            # Stabilise and exponentiate
            maxLW = maximum(logW)
            if !isfinite(maxLW)
                P = β
            else
                W = exp.(logW .- maxLW)
                P = W ./ sum(W)
            end

            # Resample the state
            S[t] = rand(Categorical(P))
            m[S[t]] += 1
            if t > 1
                n_mat[S[t-1], S[t]] += 1
            end
            if t < T
                n_mat[S[t], S[t+1]] += 1
            end
        end

        # Update (μ, Σ)
        for k in 1:K
            if m[k] > 0
                idx    = findall(==(k), S)
                Y_k    = Y[idx, :]
                Y_bar   = vec(sum(Y_k, dims=1)) ./ m[k]
                diff_mu = Y_bar .- mu0
                centered = Y_k .- Y_bar'
                S_k      = centered' * centered
                
                mu_k    = (kappa0*mu0 .+ m[k]*Y_bar) ./ (kappa0 + m[k])
                kappa_k = kappa0 + m[k]
                rho_k   = rho0   + m[k]
                Psi_k   = safe_pdmat(Psi0 + S_k +
                        (kappa0*m[k])/(kappa0+m[k]) * (diff_mu * diff_mu'))
                
                Σ_mat   = rand(InverseWishart(rho_k, Psi_k))
                Σ[k]    = safe_pdmat(Σ_mat)
                μ[k]    = rand(MvNormal(mu_k, Σ[k] / kappa_k))
            else
                Σ_mat     = rand(InverseWishart(rho0, Psi0))
                Σ[k]    = safe_pdmat(Σ_mat)
                μ[k]    = rand(MvNormal(mu0, Σ[k] / kappa0))
            end
        end

        # Form CRP variables
        for k in 1:K
            for j in 1:K
                if n_mat[j, k] == 0
                    q_mat[j, k] = 0
                else
                    base = α[j] * β[k] + λ[j] * δ[j, k]
                    b     = zeros(Int, n_mat[j, k])
                    b[1]  = 1
                    for l in 2:n_mat[j, k]
                        b[l] = rand(safe_bernoulli(base / (base + l - 1)))
                    end
                    q_mat[j, k] = sum(b)
                end
            end
        end
        q_alpha_diag  = zeros(Int, K)
        q_lambda_diag = zeros(Int, K)
        for k in 1:K
            qkk = q_mat[k, k]
            if qkk > 0
                a = α[k]*β[k]; b = λ[k]
                denom = a + b
                pα = denom > 0 ? clamp(a/denom, 0.0, 1.0) : 0.5
                q_alpha_diag[k]  = rand(Binomial(qkk, pα))
                q_lambda_diag[k] = qkk - q_alpha_diag[k]
            end
        end
        for k in 1:K
            Q[k] = (sum(q_mat[:, k]) - q_mat[k, k]) + q_alpha_diag[k]
        end
        Q[S[1]] += 1
        M = sum(Q)

        # Update β
        β = safe_dirichlet(Q .+ 1)
        β .= β ./ sum(β)

        # Update ξ and χ
        for j in 1:K
            if m[j] > 0
                ξ[j] = rand(safe_beta(X1 + q_lambda_diag[j], X2 + sum(q_mat[j, :]) - q_lambda_diag[j]))
                r_j  = rand(safe_beta(χ[j] + 1, sum(n_mat[j, :])))
                numerator = C1 + sum(q_mat[j, :]) - 1
                denominator = numerator + sum(n_mat[j, :]) * (C2 - safe_log(r_j))
                z_j  = rand(safe_bernoulli(numerator/denominator))
                χ[j] = rand(safe_gamma(numerator + z_j, C2 - safe_log(r_j)))
                α[j] = (1 - ξ[j]) * χ[j]
                λ[j] = ξ[j]  * χ[j]
            else
                ξ[j] = rand(safe_beta(X1, X2))
                χ[j] = rand(safe_gamma(C1, C2))
                α[j] = (1 - ξ[j]) * χ[j]
                λ[j] = ξ[j]  * χ[j]
            end
        end

        # Update π
        for j in 1:K
            π_vec[j] = safe_dirichlet(α[j] .* β .+ λ[j] .* δ[j, :] .+ n_mat[j, :])
            π_vec[j] .= π_vec[j] ./ sum(π_vec[j])
        end 

        if n_iter > N_warmup

            S_future = zeros(Int, H)
            S_future[1] = sample(1:K, Weights(π_vec[S[T]]))
            for h in 2:H
                S_future[h] = sample(1:K, Weights(π_vec[S_future[h-1]]))
            end

            for h in 1:H
                k = S_future[h]
                if m[k] > 0
                    idx    = findall(==(k), S)
                    Y_k    = Y[idx, :]
                    Y_bar   = vec(sum(Y_k, dims=1)) ./ m[k]
                    diff_mu = Y_bar .- mu0
                    centered = Y_k .- Y_bar'
                    S_k      = centered' * centered
                    
                    mu_k    = (kappa0*mu0 .+ m[k]*Y_bar) ./ (kappa0 + m[k])
                    kappa_k = kappa0 + m[k]
                    rho_k   = rho0   + m[k]
                    Psi_k   = safe_pdmat(Psi0 + S_k +
                            (kappa0*m[k])/(kappa0+m[k]) * (diff_mu * diff_mu'))
                    
                    Σ_mat   = rand(InverseWishart(rho_k, Psi_k))
                    Σ_h    = safe_pdmat(Σ_mat)
                    μ_h    = rand(MvNormal(mu_k, Σ[k] / kappa_k))
                else
                    Σ_mat   = rand(InverseWishart(rho0, Psi0))
                    Σ_h     = safe_pdmat(Σ_mat)
                    μ_h     = rand(MvNormal(mu0, Σ_h / kappa0))
                end

                if n_iter == N_warmup + 1
                    mu_bar[h, :]       = μ_h
                    mumuT_bar[h, :, :] = μ_h * μ_h'
                    Sigma_bar[h, :, :] = Σ_h
                else
                    mu_bar[h, :]       += μ_h
                    mumuT_bar[h, :, :] += μ_h * μ_h'
                    Sigma_bar[h, :, :] += Σ_h
                end
            end
        end
    end

    mu_bar    ./= N_sample
    EΣ        = Sigma_bar    ./ N_sample
    EmumuT    = mumuT_bar    ./ N_sample
    Covμ      = zeros(H, D, D)
    for h in 1:H
        Covμ[h, :, :] = EmumuT[h, :, :] .- mu_bar[h, :]*mu_bar[h, :]' 
    end
    fullΣ = EΣ .+ Covμ

    # Package up the final sampler state for warm-starting the next day
    state_out = PPState(;
        S = copy(S),
        β = copy(β),
        ξ = copy(ξ),
        χ = copy(χ),
        α = copy(α),
        λ = copy(λ),
        π_vec = deepcopy(π_vec),
        μ = deepcopy(μ),
        Σ = deepcopy(Σ),
    )

    return (mu_bar = mu_bar, Sigma_bar = fullΣ, K = K, state = state_out)
end


run_pp (generic function with 1 method)

In [6]:
# Simulate one dataset
function simulate_series(pi_true, mu_true, Sigma_true; Total_T=1764)
    K = length(pi_true)
    Dloc = length(mu_true[1])
    S = Vector{Int}(undef, Total_T)
    Yf = Matrix{Float64}(undef, Total_T, Dloc)

    S[1] = rand(1:K)
    Yf[1, :] .= rand(MvNormal(mu_true[S[1]], Sigma_true[S[1]]))
    for t in 2:Total_T
        S[t] = rand(Categorical(pi_true[S[t-1]]))
        Yf[t, :] .= rand(MvNormal(mu_true[S[t]], Sigma_true[S[t]]))
    end
    return S, Yf
end

function backtest_once(seed::Int; Total_T=1764, lookback=504, H=20, prefix="G-2")
    Random.seed!(seed)

    # Simulate data for this seed
    _, Y_full = simulate_series(pi_true, mu_true, Sigma_true; Total_T=Total_T)
    Y      = copy(Y_full[1:lookback, :])
    Y_test = copy(Y_full[lookback+1:end, :])

    # Dimensions
    D   = size(Y, 2)
    T   = lookback
    n_days = size(Y_test, 1)

    # Starting portfolio state
    x = fill(1/D, D)
    h = fill(1000/D, D)
    V = [sum(h)]
    days_traded = 0

    # Drawdown / risk settings and costs
    gamma0   = 4.0
    drawdown = 60
    D_max    = 0.08
    kappa    = fill(0.001, D)

    # Per-test CSV
    cd(joinpath(@__DIR__, "G-2"))
    test_csv = "$(prefix) $(seed).csv"
    open(test_csv, "w") do io
        hdr = ["iter", "final_K", "fees", "V_new"]
        append!(hdr, string.("x", 1:D))
        println(io, join(hdr, ","))
    end

    pp_state = nothing

    for day in 1:n_days
        # Run sampler
        pp = run_pp(Matrix(Y), T, H, D; state=pp_state)

        mu_bar, Sigma_bar, K = pp.mu_bar, pp.Sigma_bar, pp.K
        pp_state = pp.state
    
        # Drawdown-aware risk aversion
        epsM = 1e-12
        function dd_gamma(M_at, V_t; gamma0, D_max)
            D_rt = (M_at - V_t) / max(M_at, epsM)
            return gamma0 * D_max / max(D_max - D_rt, 1e-5)
        end
    
        gamma_vec = zeros(H)
        nV = length(V)
        for t in 1:H
            idx = clamp(nV - (t - 1), 1, nV)
            s   = max(1, idx - drawdown + 1)
            M_at = maximum(@view V[s:idx])
            gamma_vec[t] = dd_gamma(M_at, V[idx]; gamma0=gamma0, D_max=D_max)
        end
    
        prev = copy(x)
    
        # Two-step optimisation
        # Build risk tensors
        Theta_true = [Sigma_bar[t, :, :] for t in 1:H]
        Theta_psd  = [Sigma_bar[t, :, :] for t in 1:H]
        
        # Convex first solve
        function solve_model_simple(Theta::Vector{Matrix{Float64}}; 
                                    X_start::Union{Nothing,Matrix{Float64}}=nothing,
                                    fee_cap_bps::Float64 = 2.5,
                                    cap_all_h::Bool = false)
            model = Model(Ipopt.Optimizer)
            set_silent(model)
    
            @variable(model, -0.3 <= x_var[1:D, 1:H] <= 0.5)
            @constraint(model, [t=1:H], sum(x_var[i,t] for i in 1:D) == 1)
    
            # Turnover variables
            @variable(model, y[1:D, 1:H] >= 0)
    
            # t = 1
            @constraint(model, [i=1:D], y[i,1] >=  x_var[i,1] - prev[i])
            @constraint(model, [i=1:D], y[i,1] >= -x_var[i,1] + prev[i])
    
            # t ≥ 2
            @constraint(model, [i=1:D, t=2:H], y[i,t] >=  x_var[i,t] - x_var[i,t-1])
            @constraint(model, [i=1:D, t=2:H], y[i,t] >= -x_var[i,t] + x_var[i,t-1])
    
            # Daily fee cap
            fee_cap = fee_cap_bps / 10000.0
            if cap_all_h
                @constraint(model, [t=1:H], sum(kappa[i] * y[i,t] for i in 1:D) <= fee_cap + 1e-10)
            else
                @constraint(model, sum(kappa[i] * y[i,1] for i in 1:D) <= fee_cap + 1e-10)
            end
    
            # Objective
            @expression(model, return_term, sum(mu_bar[t,i] * x_var[i,t] for t in 1:H, i in 1:D))
            @expression(model, risk_term,   sum(gamma_vec[t] * sum(Theta[t][i,j] * x_var[i,t] * x_var[j,t]
                                                for i in 1:D, j in 1:D) for t in 1:H))
            @expression(model, tc_term,     sum(kappa[i] * y[i,t] for i in 1:D, t in 1:H))
            @objective(model, Max, return_term - risk_term - tc_term)
    
            if X_start !== nothing
                for t in 1:H, i in 1:D
                    set_start_value(x_var[i,t], X_start[i,t])
                end
            end
    
            optimize!(model)
            st = termination_status(model)
            if !(st in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED))
                return hcat([t == 1 ? copy(prev) : copy(prev) for t in 1:H]...)
            end
            sol = value.(x_var)
            hcat([sol[:,t] for t in 1:H]...)
        end
    
        # Full second solve
        function solve_model_full(Theta::Vector{Matrix{Float64}}; 
                                X_start::Union{Nothing,Matrix{Float64}}=nothing,
                                fee_cap_bps::Float64 = 2.5,
                                cap_all_h::Bool = false)
            model = Model(Ipopt.Optimizer)
            set_silent(model)
    
            @variable(model, -0.3 <= x_var[1:D, 1:H] <= 0.5)
            @constraint(model, [t=1:H], sum(x_var[i,t] for i in 1:D) == 1)
    
            @variable(model, y[1:D, 1:H] >= 0)
    
            # t = 1
            @constraint(model, [i=1:D], y[i,1] >=  x_var[i,1] - prev[i])
            @constraint(model, [i=1:D], y[i,1] >= -x_var[i,1] + prev[i])
    
            # t ≥ 2
            @expression(model, denom[t=2:H], sum(x_var[j,t-1] * (1 + mu_bar[t-1,j]) for j in 1:D))
            @variable(model, w[1:D, 2:H])
            @constraint(model, [t=2:H], sum(w[i,t] for i in 1:D) == 1)
            @NLconstraint(model, [i=1:D, t=2:H], w[i,t] * denom[t] == x_var[i,t-1] * (1 + mu_bar[t-1,i]))
            @constraint(model, [i=1:D, t=2:H], y[i,t] >=  x_var[i,t] - w[i,t])
            @constraint(model, [i=1:D, t=2:H], y[i,t] >= -x_var[i,t] + w[i,t])
    
            # Daily fee cap
            fee_cap = fee_cap_bps / 10000.0
            if cap_all_h
                @constraint(model, [t=1:H], sum(kappa[i] * y[i,t] for i in 1:D) <= fee_cap + 1e-10)
            else
                @constraint(model, sum(kappa[i] * y[i,1] for i in 1:D) <= fee_cap + 1e-10)
            end
    
            # Objective
            @expression(model, return_term, sum(mu_bar[t,i] * x_var[i,t] for t in 1:H, i in 1:D))
            @expression(model, risk_term,   sum(gamma_vec[t] * sum(Theta[t][i,j] * x_var[i,t] * x_var[j,t]
                                                for i in 1:D, j in 1:D) for t in 1:H))
            @expression(model, tc_term,     sum(kappa[i] * y[i,t] for i in 1:D, t in 1:H))
            @objective(model, Max, return_term - risk_term - tc_term)
    
            if X_start !== nothing
                for t in 1:H, i in 1:D
                    set_start_value(x_var[i,t], X_start[i,t])
                end
            end
    
            optimize!(model)
            st = termination_status(model)
            if !(st in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED))
                return hcat([t == 1 ? copy(prev) : copy(prev) for t in 1:H]...)
            end
            sol = value.(x_var)
            hcat([sol[:,t] for t in 1:H]...)
        end
    
        # Pick the better under the true objective (in case second solve gave worse solution)
        best_X(X_init, X_opt, Theta_true; mu_bar, gamma_vec, kappa, prev) = begin
            D, H = size(X_init)
            wprev(X, t) = (d = X[:,t-1] .* (1 .+ mu_bar[t-1,:]); d ./ max(sum(d), eps()))
            f = X -> begin
                ret  = sum(mu_bar[t,:]' * X[:,t]                             for t in 1:H)
                risk = sum(gamma_vec[t] * (X[:,t]' * Theta_true[t] * X[:,t]) for t in 1:H)
                tc   = sum(kappa .* abs.(X[:,1] .- prev)) +
                    sum(sum(kappa .* abs.(X[:,t] .- wprev(X,t)))          for t in 2:H)
                ret - risk - tc
            end
            f(X_opt) ≥ f(X_init) ? X_opt : X_init
        end
    
        X_init = solve_model_simple(Theta_psd;  fee_cap_bps=2.5, cap_all_h=false)
        X_opt  = solve_model_full(  Theta_true; X_start=X_init, fee_cap_bps=2.5, cap_all_h=false)
        X_final = best_X(X_init, X_opt, Theta_true; mu_bar=mu_bar, gamma_vec=gamma_vec, kappa=kappa, prev=prev)
        x_target = X_final[:,1]
        
        # Rebalance with costs applied before returns
        V_pre    = sum(h)
        h_raw    = x_target * V_pre
        total_tc = sum(kappa .* abs.(h_raw .- h))
        V_post   = V_pre - total_tc
        
        h = x_target * V_post
        x = h / V_post
        
        # Apply next-day simple returns
        r_next = vec(Y_test[1, :])
        h      .= h .* (1 .+ r_next)
        
        V_new   = sum(h)
        x      .= h ./ V_new
        
        # Roll Y / Y_test
        Y      = vcat(Y[2:end, :], Y_test[1:1, :])
        Y_test = Y_test[2:end, :]
        
        # Record
        days_traded += 1
        push!(V, V_new)
    
        row = vcat([days_traded, K, total_tc, V_new], x_target)
        open(test_csv, "a") do io
            println(io, join(row, ","))
        end
    end

    cd(@__DIR__)

    return V[end]
end

summary_csv = "G-2 summary.csv"
open(summary_csv, "w") do io
    println(io, "test,final_V")
end

# Run 25 independent tests
for test in 1:25
    final_V = backtest_once(test; Total_T=1764, lookback=504, H=20, prefix="G-2")
    open(summary_csv, "a") do io
        println(io, "$(test),$(final_V)")
    end
end